In [1]:
using IntervalArithmetic, LinearAlgebra, Serialization

In [2]:
setprecision(1024)

LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))

function enlarge(x::Interval{BigFloat})
    isguaranteed(x) || error("interval is not guaranteed")
    return interval(BigFloat(inf(x), RoundDown), BigFloat(sup(x), RoundUp))
end

function Base.inv(P::Bidiagonal{Interval{BigFloat}, Vector{Interval{BigFloat}}})
    if P.uplo == 'U'
        C = -P.ev./P.dv[1:end-1]
        invC = UpperTriangular(zeros(eltype(P), size(P)))
        for i in 1:size(P)[1]
            invC[i,i:end] = cumprod([one(eltype(P)); C[i:end]])
        end
        return UpperTriangular(invC ./ P.dv')
    else
        return LowerTriangular(inv(P')')
    end
end    

function Base.:*(D::Diagonal, v::Vector)
    return diag(D).*v
end

function Base.:*(D::Diagonal, A::Matrix)
    return diag(D).*A
end

function Base.:*(A::Matrix, D::Diagonal)
    return A.*diag(D)'
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        all(isguaranteed.(A)) || error("matrix not guaranteed")
        return sqrt(interval(maximum(sup.(sum(abs.(A), dims = 1))))*interval(maximum(sup.(sum(abs.(A), dims = 2)))))
    end
end

op_norm (generic function with 1 method)

In [3]:
ω = interval(8)
n = 2500
setprecision(8192)
b₁ = deserialize("b1_k4")
setprecision(precision(mid.(b₁)))
b₀ = interval(BigFloat, 0.0)
b = zeros(Interval{BigFloat},n+3)
b[1:2] = [b₀, b₁]
for k = 2:n+2
    b[k+1] = interval(4)+(interval(BigFloat, k-1)/b[k])-b[k]-b[k-1]
end
setprecision(1024)
b = b[2:end]
a = enlarge.(sqrt.(b))
b = enlarge.(b);

In [4]:
α = interval.(BigFloat, collect(2:2:n))./a[2:2:n]
β = a[2:2:n].*a[3:2:n+1].*a[4:2:n+2]
P̄ = Bidiagonal([interval(big(1)); α], [interval(big(0)); β[1:end-1]] , :U)
P̄⁻¹ = inv(P̄);
Π₀ = Diagonal([interval(1); zeros(Interval{Float64}, n÷2)])
λ = α.^2 + β.^2
μ = α[2:end].*β[1:end-1];

In [5]:
C_α = deserialize("C_alpha")
θ = deserialize("theta");
Cₚ₀ = deserialize("CP0");
c₀ = deserialize("c0");
C₁₂ = β[end]*norm(P̄⁻¹[:,end])/C_α/sqrt(interval(1)-θ^2)
C₂₂ = interval(1)/C_α/(interval(1)-θ)
Cd = interval(1)/C_α/sqrt(interval(1)-θ^2)
C = sqrt(C₁₂^2+C₂₂^2);
Z = enlarge(deserialize("GP_Z"))

[103.5, 103.501]₁₀₂₄_com

In [6]:
V4p = deserialize("GP_V4");
V6p = deserialize("GP_V6");
fV4p = interval.(Float64, deserialize("GP_V4"));
fV6p = interval.(Float64, deserialize("GP_V6"));
V4q = fV4p*interval.(Float64, P̄⁻¹);
V6q = fV6p*interval.(Float64, P̄⁻¹);

In [7]:
ūq = interval.(deserialize("ubar2"));
ūp = P̄⁻¹*ūq;

In [8]:
Πₙdfū = ω*interval.(Float64,P̄⁻¹) + Π₀ - interval(3)*(fV4p'*(((fV4p*interval.(Float64,ūp)).^2).*V4q));

In [9]:
Πₙfū = ω*ūp+Π₀*ūq-V4p'*((V4p*ūp).^3);

In [10]:
ΠₙFū = ūq - P̄⁻¹'*Πₙfū;

In [11]:
DFū = interval.(I(n÷2+1)) - interval.(Float64,P̄⁻¹)'*Πₙdfū;

In [12]:
Aₙ = interval.(inv(Float64.(mid.(DFū))));

In [13]:
Z¹¹ = op_norm(interval.(I(n÷2+1)) - Aₙ * interval.(Float64, DFū))

[7.2352e-12, 7.23521e-12]_com

In [14]:
Y = sqrt(norm(Aₙ*ΠₙFū)^2 + (β[end]*Cd*abs(P̄⁻¹[:,end]'*Πₙfū)+C₂₂*sqrt(sum((V6p*ūp).^6) - sum((V4p'*((V4p*ūp).^3)).^2)))^2/interval(n)^interval(3//2))

[2.67869e-142, 2.6787e-142]₁₀₂₄_com

In [15]:
wp = vec(sqrt.(abs.(sum((((fV6p*interval.(Float64,ūp)).^2).*fV6p).^2, dims=1) - sum((fV4p'*(((fV4p*interval.(Float64,ūp)).^2).*fV4p)).^2, dims = 1))));

In [16]:
wq = vec(sqrt.(abs.(sum((((fV6p*interval.(Float64,ūp)).^2).*V6q).^2, dims=1) - sum((fV4p'*(((fV4p*interval.(Float64,ūp)).^2).*V4q)).^2, dims = 1))));

In [17]:
Z²¹ = (β[end]*Cd*norm(vec(P̄⁻¹[:,end]'*Πₙdfū))+interval(3)*C₂₂*norm(wq))/interval(n)^interval(3//4)

[1.91831e-05, 1.91833e-05]₁₀₂₄_com

In [18]:
Z¹² = (β[end]*Cd*norm(Aₙ*(P̄⁻¹'*Πₙdfū[:,end]))+interval(3)*C₂₂*norm(Aₙ*(P̄⁻¹'*wp)))/interval(n)^interval(3//4)

[1.81176e-05, 1.81181e-05]₁₀₂₄_com

In [19]:
supφ̄ = sqrt(Z*(interval(1)+c₀))*sqrt(norm(ūq))*sqrt(norm(ūp))

[2.31465, 2.31466]₁₀₂₄_com

In [20]:
Z²² = C^2*(interval(3)*supφ̄^2+abs(ω))/interval(n)^interval(3//2)

[0.000288058, 0.00028806]₁₀₂₄_com

In [21]:
Z₁ = op_norm([Z¹¹ Z¹²;
        Z²¹ Z²²])

[0.000289262, 0.000289263]₁₀₂₄_com

In [22]:
norm_A = max(op_norm(Aₙ), interval(1))

[84.5426, 84.5427]_com

In [23]:
Z₂ = interval(6)*Z*(interval(1)+c₀)*norm_A*max(interval(1), Cₚ₀)^interval(3//2)*norm(ūp)

[16920.5, 16920.6]₁₀₂₄_com

In [24]:
Z₃ = interval(6)*Z*(interval(1)+c₀)*norm_A*max(interval(1), Cₚ₀)^interval(3//2)

[296317.0, 296318.0]₁₀₂₄_com

In [25]:
P(δ) = Z₃/interval(6)*δ^3 + Z₂/interval(2)*δ^2 - (interval(1) - Z₁)*δ + Y

P (generic function with 1 method)

In [26]:
δ̲ = Y/(interval(1)-Z₁)*(interval(1)+interval(BigFloat,2)^(-48))
if sup(P(δ̲))<0
    println("δ̲ validated")
end

δ̲ validated


In [27]:
δ̲

[2.67946e-142, 2.67947e-142]₁₀₂₄_com

In [28]:
η = sqrt(Z)*(max(interval(1), Cₚ₀)+(interval(1)+c₀)^2/interval(4))^interval(1//2)*δ̲

[8.16138e-141, 8.16139e-141]₁₀₂₄_com